In [1]:
import pandas as pd
import numpy as np
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os, glob
import joblib
import random
import math
from tqdm import tqdm 
from collections import OrderedDict

from scipy.interpolate import interp1d
from scipy import signal
from scipy.signal import argrelmax

from sklearn.metrics import mean_squared_error

from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torch.optim import AdamW, RAdam
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler

from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa 
import ctypes

import sys
sys.path.append("detw")


from engine import train_one_epoch, evaluate
import utils

import timm

/home/itsuki/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
timm.__version__

'0.9.11'

In [3]:
model_dir = './exp57'
os.makedirs(model_dir, exist_ok=True)

In [4]:
# Fundamental config
WORKERS = os.cpu_count() // 2
N_FOLDS = 5
TRAIN_FOLD = 0
MAX_LEN = 2880
USE_AMP = True
SEED = 8620

In [5]:
IMG_SIZE = (384, MAX_LEN)
HEIGHT = IMG_SIZE[0]
WIDTH = IMG_SIZE[1]

In [6]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [7]:
# Train config
EPOCHS = 50
BS = 3

In [8]:
# Optimizer config
LR = 1e-4 * BS / 8
WD = 1e-6

In [9]:
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.use_deterministic_algorithms = True
    torch.backends.cudnn.benchmark = True

torch_fix_seed(SEED)

In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [11]:
class SleepDataset(Dataset):
    def __init__(
        self,
        X_folder,
        y_folder,
        max_len=2**12,
        is_train=False,
        sample_per_epoch=None,
        win_size=180,
    ):
        self.enmo_mean = np.load('enmo_mean.npy')
        self.enmo_std = np.load('enmo_std.npy')
        
        self.max_len = max_len
        self.win_size = win_size
        
        self.is_train = is_train
        
        self.sample_per_epoch = sample_per_epoch
        
        self.feat_list = ['anglez','enmo']
        
        self.Xs = self.read_csvs(X_folder)
        self.events = self.read_events(y_folder)
        
        self.label_list = ['onset', 'wakeup']
        
        self.hour_feat= ['hour']
        
    def read_csvs(self, folder):
        res = []
        if type(folder) is str:
            files = sorted(glob.glob(f'{folder}/*.csv'))
        else:
            files = folder
        for i, f in tqdm(enumerate(files), total=len(files), leave=True):
            df = pd.read_csv(f)
            df = self.norm_feat_eng(df, init=True if i==0 else False)
            res.append(df)
        return res

    def read_events(self, folder):
        res = []
        if type(folder) is str:
            files = sorted(glob.glob(f'{folder}/*.csv'))
        else:
            files = folder
        for i, f in tqdm(enumerate(files), total=len(files), leave=False):
            df = pd.read_csv(f)[['event', 'step']]
            res.append(df)
        return res

    def norm_feat_eng(self, X, init=False):
        X['anglez'] = X['anglez'] / 90.0
        X['enmo_ln1p'] = np.log1p(X['enmo'])
        X['enmo'] = (X['enmo']-self.enmo_mean)/ (self.enmo_std+1e-12)

        X['anglez_2'] = X['anglez'] ** 2 
    
        X['enmo_ln1p_2'] = X['enmo_ln1p'] ** 2
        X['enmo_ln1p_05'] = X['enmo_ln1p'] ** 0.5
        X['enmo_ln1p_4'] = X['enmo_ln1p'] ** 4

        if init:
            self.feat_list.append('anglez_2')
            self.feat_list.append('enmo_ln1p_2')
            self.feat_list.append('enmo_ln1p_05')
            self.feat_list.append('enmo_ln1p_4')
            
            self.feat_list.append('enmo_ln1p')

        f = X['anglez']
        g = X['enmo']
        h = X['enmo_ln1p']
        n_grads = 2
        for i in range(n_grads):
            f = np.gradient(f)
            g = np.gradient(g)
            h = np.gradient(h)
            X['anglez_grad_' + str(i+1)] = f
            X['enmo_grad_' + str(i+1)] = g
            X['enmo_ln1p_grad_' + str(i+1)] = h
            if init:
                self.feat_list.append('anglez_grad_' + str(i+1))
                self.feat_list.append('enmo_grad_' + str(i+1))
                self.feat_list.append('enmo_ln1p_grad_' + str(i+1))
       
        for w in [1, 2, 4, 8, 16, 32]:    
            X['anglez_shift_pos_' + str(w)] = X['anglez'].shift(w).fillna(0)
            X['anglez_shift_neg_' + str(w)] = X['anglez'].shift(-w).fillna(0)
            
            X['enmo_shift_pos_' + str(w)] = X['enmo'].shift(w).fillna(0)
            X['enmo_shift_neg_' + str(w)] = X['enmo'].shift(-w).fillna(0)

            X['enmo_ln1p_shift_pos_' + str(w)] = X['enmo_ln1p'].shift(w).fillna(0)
            X['enmo_ln1p_shift_neg_' + str(w)] = X['enmo_ln1p'].shift(-w).fillna(0)            
            
            if init:
                self.feat_list.append('anglez_shift_pos_' + str(w))
                self.feat_list.append('anglez_shift_neg_' + str(w))
                
                self.feat_list.append('enmo_shift_pos_' + str(w))
                self.feat_list.append('enmo_shift_neg_' + str(w))

                self.feat_list.append('enmo_ln1p_shift_pos_' + str(w))
                self.feat_list.append('enmo_ln1p_shift_neg_' + str(w))
            
        for r in [5, 17, 33, 65, 129]:
            tmp_anglez = X['anglez'].rolling(r, center=True)
            X[f'anglez_mean_{r}'] = tmp_anglez.mean()
            X[f'anglez_std_{r}'] = tmp_anglez.std()
            
            tmp_enmo = X['enmo'].rolling(r, center=True)
            X[f'enmo_mean_{r}'] = tmp_enmo.mean()
            X[f'enmo_std_{r}'] = tmp_enmo.std()

            tmp_enmo_ln1p = X['enmo_ln1p'].rolling(r, center=True)
            X[f'enmo_ln1p_mean_{r}'] = tmp_enmo_ln1p.mean()
            X[f'enmo_ln1p_std_{r}'] = tmp_enmo_ln1p.std()
            
            if init:
                self.feat_list.append(f'anglez_mean_{r}')
                self.feat_list.append(f'anglez_std_{r}')

                self.feat_list.append(f'enmo_mean_{r}')
                self.feat_list.append(f'enmo_std_{r}')

                self.feat_list.append(f'enmo_ln1p_mean_{r}')
                self.feat_list.append(f'enmo_ln1p_std_{r}')

        X = X.fillna(0)
        return X.astype(np.float32)
    
    def __len__(self):
        return self.sample_per_epoch if self.sample_per_epoch is not None else len(self.Xs)

    def __getitem__(self, index):
        if self.is_train:
            index = torch.randint(low=0, high=len(self.Xs), size=(1,))[0]

        elif not self.is_train and len(self.Xs)!=self.__len__():
            index = torch.randint(low=0, high=len(self.Xs), size=(1,))[0]
            
        Xy = self.Xs[index]
        ev = self.events[index].copy()
        
        X = Xy[self.feat_list].values.astype(np.float32)
        t = Xy[self.hour_feat].values.astype(np.int32)

        if len(Xy)+1<self.max_len:
            res = self.max_len - len(Xy) + 1
            X = np.pad(X, ((0, res), (0, 0)))
            t = np.pad(t, ((0, res), (0, 0)), constant_values=t[-1])

        if self.is_train:
            start = torch.randint(low=0, high=len(X)-self.max_len, size=(1,))[0]

            X = X[start:start+self.max_len]
            t = t[start:start+self.max_len] 

        else:
            start = torch.randint(low=0, high=len(X)-self.max_len, size=(1,))[0]
            X = X[start:start+self.max_len]
            t = t[start:start+self.max_len] 

        ev['step'] = ev['step'] - start.numpy()
        ev = ev[(ev['step']>=0) & (ev['step']<self.max_len)].reset_index()

        boxes = []
        label_ids = []
        
        if len(ev) > 0:
            for e, s in zip(ev['event'], ev['step']):
                if e == 'onset':
                    label_ids.append(1)
                else:
                    label_ids.append(2)

                xmin = max(0, s - self.win_size)
                xmax = min(self.max_len, s + self.win_size)
                ymin = 0
                ymax = HEIGHT
                boxes.append([xmin, ymin, xmax, ymax])
            iscrowd = torch.zeros((len(ev),), dtype=torch.int32)

        else:
            label_ids.append(3)
            xmin = 0
            xmax = self.max_len
            ymin = 0
            ymax = HEIGHT
            boxes.append([xmin, ymin, xmax, ymax])
            iscrowd = torch.zeros((1,), dtype=torch.int32)

        boxes_tns = torch.tensor(boxes, dtype=torch.float32)   
        image_id = torch.tensor([index])
        area = (boxes_tns[:, 3] - boxes_tns[:, 1]) * (boxes_tns[:, 2] - boxes_tns[:, 0])

        labels = torch.tensor(label_ids)

        target = {}
        target["boxes"] = boxes_tns
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        X = np.concatenate([X,t], axis=1)
        X = torch.tensor(X)
        
        return X, target

In [12]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from typing import Any, Dict, List, Optional, Tuple
from torchvision.models.detection.image_list import *

In [13]:
def seq_block(in_c, out_c, ksh=4, usf=2, ksw=3):
    padw = (ksw-1)//2
    padh = (ksh-usf)// 2
    return nn.Sequential(
        nn.BatchNorm2d(in_c),
        nn.LeakyReLU(0.2),
        nn.ConvTranspose2d(in_c, out_c, kernel_size=(ksh,ksw), stride=(usf,1), padding=(padh,padw)),
        nn.BatchNorm2d(out_c),
        nn.LeakyReLU(0.2),
        nn.Conv2d(out_c, out_c, ksw, padding=padw)
    )
class Feat2Img(nn.Module):
    def __init__(self, in_c=2, hr_feat=32, base=16, ksw=3):
        super(Feat2Img, self).__init__()
        self.hr_emb = nn.Embedding(24, hr_feat)
        self.fc1_hr = nn.Linear(hr_feat, hr_feat)
        self.fc2_hr = nn.Linear(hr_feat, hr_feat)
        
        self.inter_conv = nn.Conv1d(in_c+hr_feat, base*128, kernel_size=1, padding=0)
        
        usfs = [3, 2, 2, 2, 2, 2, 2, 2]
        kshs = [3, 4, 4, 4, 4, 4, 4, 4]
        hid_in = [base*128, base*64, base*32, base*16, base*8, base*4, base*2, base]
        hid_out = [base*64, base*32, base*16, base*8, base*4, base*2, base, 3]
        padw = (ksw-1)//2

        self.blks = nn.Sequential(
            *[seq_block(hid_in[i], hid_out[i], kshs[i], usfs[i]) for i in range(len(hid_in))]
            )
        self.shortcuts = nn.Sequential(
            *[nn.ConvTranspose2d(hid_in[i], hid_out[i], kernel_size=(kshs[i],ksw), stride=(usfs[i], 1), padding=((kshs[i]-usfs[i])//2,padw)) for i in range(len(hid_in))]
            )
        self.relu = nn.LeakyReLU(0.2)
    
    def forward(self, x, t=None):
        x = torch.stack(x,dim=0)
        x, h = x[...,:-1].float(), x[...,-1].long()
        e = self.hr_emb(h)
        e = self.relu(e)
        e = self.fc1_hr(e)
        e = self.relu(e)
        e = self.fc2_hr(e)
        x = torch.cat([x, e], dim=-1).permute(0, 2, 1)
        x = self.inter_conv(x)
        x = x.unsqueeze(2)
        
        for s, b in zip(self.shortcuts, self.blks):
            x = s(x) + b(x)

        image_list = ImageList(x, [[HEIGHT, WIDTH]] * x.size(0))
        return image_list, t

    def postprocess(
        self,
        result: List[Dict[str, Tensor]],
        image_shapes: List[Tuple[int, int]],
        original_image_sizes: List[Tuple[int, int]],
    ) -> List[Dict[str, Tensor]]:
        return result

In [14]:
import timm
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
def get_model_convnextv2(in_c, mn='convnextv2_base.fcmae_ft_in22k_in1k_384', pretrained=True):

    backbone = timm.create_model(mn, pretrained=pretrained)


    backbone.out_channels = 1280
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                     aspect_ratios=((0.5, 1.0, 2.0),))    
        
    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                    num_classes=4,
                    rpn_anchor_generator=anchor_generator
                      )

    model.transform = Feat2Img(in_c)

    if 'large' in mn:
        model.backbone.head = nn.Conv2d(1536, 1280, kernel_size=(1, 1), stride=(1, 1), bias=True)

    elif 'base' in mn:
        model.backbone.head = nn.Conv2d(1024, 1280, kernel_size=(1, 1), stride=(1, 1), bias=True)

    else:
        assert 0  


    return model

In [15]:
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=16384)

In [16]:
all_csvs = sorted(glob.glob('train_csvs/*.csv'))
all_objs = sorted(glob.glob('train_csvs_objdet/*.csv'))

In [17]:
print(device)

all_imgs = np.array(all_csvs)
all_objs = np.array(all_objs)

dataset_train = SleepDataset(all_imgs, all_objs, MAX_LEN, is_train=True, sample_per_epoch=8000, win_size=120)
train_dl = torch.utils.data.DataLoader(
    dataset_train, batch_size=BS, shuffle=True, num_workers=os.cpu_count()//2, pin_memory=True, drop_last=True, collate_fn=utils.collate_fn)

np.save(model_dir+'/feature_list.npy', dataset_train.feat_list)

model = get_model_convnextv2(in_c=len(dataset_train.feat_list), mn='convnextv2_base.fcmae_ft_in22k_in1k_384', pretrained=True)

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
n_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
print(f'input_dim:{len(dataset_train.feat_list)}, params:{n_params}')
optimizer = RAdam(params, lr=LR, weight_decay=WD)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

best_loss = 1e9

for epoch in range(EPOCHS):
    model.train()
    train_one_epoch(model, optimizer, train_dl, device, epoch, print_freq=200, scaler=scaler)

    scheduler.step()

    if epoch%5==0 and epoch>=10:
        if device!='cuda:0':
            model.to('cuda:0')
        model_path = f'{model_dir}/fepoch{epoch}.pth'
        torch.save(model.state_dict(), model_path)
        if device!='cuda:0':
            model.to(device)

if device!='cuda:0':
    model.to('cuda:0')
model_path = f'{model_dir}/final.pth'
torch.save(model.state_dict(), model_path)
if device!='cuda:0':
    model.to(device)

gc.collect()
torch.cuda.empty_cache()


cuda:0


100%|█████████████████████████████████████████| 277/277 [01:51<00:00,  2.49it/s]


input_dim:79, params:236497935
Epoch: [0]  [   0/2666]  eta: 8:21:15  lr: 0.000000  loss: 2.1754 (2.1754)  loss_classifier: 1.4052 (1.4052)  loss_box_reg: 0.0087 (0.0087)  loss_objectness: 0.7375 (0.7375)  loss_rpn_box_reg: 0.0239 (0.0239)  time: 11.2811  data: 4.7396  max mem: 18968
Epoch: [0]  [ 200/2666]  eta: 0:21:43  lr: 0.000008  loss: 0.2090 (0.8789)  loss_classifier: 0.0957 (0.4328)  loss_box_reg: 0.0635 (0.0313)  loss_objectness: 0.0386 (0.3972)  loss_rpn_box_reg: 0.0074 (0.0176)  time: 0.4754  data: 0.0001  max mem: 21458
Epoch: [0]  [ 400/2666]  eta: 0:18:56  lr: 0.000015  loss: 0.1285 (0.5215)  loss_classifier: 0.0758 (0.2617)  loss_box_reg: 0.0347 (0.0395)  loss_objectness: 0.0166 (0.2091)  loss_rpn_box_reg: 0.0031 (0.0113)  time: 0.4759  data: 0.0001  max mem: 21458
Epoch: [0]  [ 600/2666]  eta: 0:16:59  lr: 0.000023  loss: 0.1000 (0.3873)  loss_classifier: 0.0518 (0.1958)  loss_box_reg: 0.0270 (0.0381)  loss_objectness: 0.0151 (0.1445)  loss_rpn_box_reg: 0.0043 (0.0090) 